### Document Loaders

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./bella_vista.txt")
docs = loader.load()

In [3]:
print(docs)
print(len(docs))

[Document(metadata={'source': './bella_vista.txt'}, page_content="Q: What are the hours of operation for Bella Vista?\nA: Bella Vista is open from 11 a.m. to 11 p.m. from Monday to Saturday. On Sundays, we welcome guests from 12 p.m. to 10 p.m.\n\nQ: What type of cuisine does Bella Vista serve?\nA: Bella Vista offers a delightful blend of Mediterranean and contemporary American cuisine. We pride ourselves on using the freshest ingredients, many of which are sourced locally.\n\nQ: Do you offer vegetarian or vegan options at Bella Vista?\nA: Absolutely! Bella Vista boasts a diverse menu that includes a variety of vegetarian and vegan dishes. Our chefs are also happy to customize dishes based on dietary needs.\n\nQ: Is Bella Vista family-friendly?\nA: Yes, Bella Vista is a family-friendly establishment. We have a dedicated kids' menu and offer high chairs and booster seats for our younger guests.\n\nQ: Can I book private events at Bella Vista?\nA: Certainly! Bella Vista has a private dini

In [4]:
from langchain.schema import Document

example_doc = Document(page_content="test", metadata={"important_info": "hi there"})
example_doc

Document(metadata={'important_info': 'hi there'}, page_content='test')

Texts are not loaded 1:1 into the database, but in pieces, so called "chunks". You can define the chunk size and the overlap between the chunks.

To create multiple documents (chunks), you can use a text splitter.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
documents = text_splitter.split_documents(docs)

In [6]:
for doc in documents:
    print(doc)

len(documents)

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='A: Bella Vista is open from 11 a.m. to 11 p.m. from Monday to Saturday. On Sundays, we welcome' metadata={'source': './bella_vista.txt'}
page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.' metadata={'source': './bella_vista.txt'}
page_content='Q: What type of cuisine does Bella Vista serve?' metadata={'source': './bella_vista.txt'}
page_content='A: Bella Vista offers a delightful blend of Mediterranean and contemporary American cuisine. We' metadata={'source': './bella_vista.txt'}
page_content='cuisine. We pride ourselves on using the freshest ingredients, many of which are sourced locally.' metadata={'source': './bella_vista.txt'}
page_content='Q: Do you offer vegetarian or vegan options at Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='A: Absolutely! Bella Vista boasts a diverse menu that includes a variety of vegetarian and veg

22

### Embeddings

Texts are not stored as text in the database, but as vector representations. 
Embeddings are a type of word representation that represents the semantic meaning of words in a vector space.

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [8]:
embedding1 = embeddings.embed_query(text="The solar system consists of the Sun and the objects that orbit it")
print(embedding1)
print(len(embedding1))

[0.025021199136972427, 0.021153723821043968, -0.010465656407177448, -0.02232755906879902, -0.04045402631163597, 0.022426407784223557, -0.005470684263855219, -0.016285402700304985, -0.0011846456909552217, -0.009279467165470123, 0.008952029049396515, 0.00534403370693326, -0.014419623650610447, 0.0019661718979477882, -0.0061039370484650135, -0.015544033609330654, 0.011071108281612396, -0.004148576408624649, 0.006425196770578623, -0.0035925498232245445, 0.002349212532863021, -0.007043004035949707, -0.011651846580207348, -0.00018891390936914831, 0.017298607155680656, -0.004201089963316917, 0.011719806119799614, -0.03195299580693245, 0.022895941510796547, -0.009328891523182392, 0.02027643844485283, -0.010725135914981365, -0.019831616431474686, -0.015506965108215809, 0.01569230668246746, 0.004111507907509804, 0.007203633897006512, 0.010132040828466415, -0.03210126981139183, 0.004204179160296917, 0.011417080648243427, 0.03368285670876503, -0.004624288063496351, -0.0052235615439713, -0.01261562

In [9]:
embedding2 = embeddings.embed_query(text="The solar system consists of the Sun and the objects that orbit it")
embedding3 = embeddings.embed_query(text="Planets, asteroids, and comets are part of our solar system.")
embedding4 = embeddings.embed_query(text="I love baking chocolate chip cookies on weekends.")

In [10]:
import numpy as np

def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)
    return dot_product / (norm_a * norm_b)

In [11]:
sim_1_2 = cosine_similarity(embedding1, embedding2)
sim_1_3 = cosine_similarity(embedding1, embedding3)
sim_3_4 = cosine_similarity(embedding3, embedding4)

print(sim_1_2, sim_1_3, sim_3_4)


1.0 0.9003968190389814 0.6963035596148098


### Loading Vectors into VectorDB (FAISS)

As created by OpenAIEmbeddings vectors and documents can now be stored in the database. This DB can be stored as .pkl file

In [13]:
from langchain.vectorstores.faiss import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)

vectorstore.save_local("index") # newer FAISS versions can not be serialized with pickle

### Loading the database

Before using the database, it must of course be loaded again.

In [15]:
# Argument 'allow_dangerous_deserialization' was added due to people adding malware in pkl files -> no worry
FAISS.load_local("index", embeddings,allow_dangerous_deserialization=True)

In [16]:
retriever = vectorstore.as_retriever()

In [17]:
docs = retriever.invoke(input="When are the opening hours??")
for doc in docs:
    print(doc)


page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.' metadata={'source': './bella_vista.txt'}
page_content='A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends' metadata={'source': './bella_vista.txt'}
page_content='during weekends and holidays, to ensure a seamless dining experience.' metadata={'source': './bella_vista.txt'}


In [18]:
docs = retriever.invoke(input="When are the opening hours?", filter={'source': './bella_vista.txt'}, k=3)
for doc in docs:
    print(doc) # does not work!

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.' metadata={'source': './bella_vista.txt'}
page_content='A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends' metadata={'source': './bella_vista.txt'}


In [19]:
retriever = vectorstore.as_retriever(search_kwargs={"filter": {'source': './bella_vista.txt'}, "k":1})
docs = retriever.invoke(input="When are the opening hours??")
for doc in docs:
    print(doc)

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}


### Now we have to pass the documents to an LLM.

We create a prompt with a question and context. Context is the output from the retriever (Document Store).
LangChain provides chains out-of-the-box to do that, the RetrievalChains

### Old way (deprecated)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant for our restaurant.

{context}

Question: {question}
Answer here:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

llm = ChatOpenAI(model="gpt-4o-mini")
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
)

result = qa.invoke(input="When are the opening hours on sunday??")
print(result)

### New Way

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant for our restaurant.

{context}

Question: {input}
Answer here:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "input"]
)


llm = ChatOpenAI(model="gpt-3.5-turbo")
combine_docs_chain = create_stuff_documents_chain(llm, PROMPT)
qa = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=combine_docs_chain
)

result = qa.invoke({"input": "When are the opening hours on sunday??"})
print(result)

{'input': 'When are the opening hours on sunday??', 'context': [Document(id='466e3083-dfa7-42a2-b7cb-33cea40d9c91', metadata={'source': './bella_vista.txt'}, page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.')], 'answer': 'The opening hours on Sunday are from 12 p.m. to 10 p.m.'}
